In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/boxoffice24/boc24.csv
/kaggle/input/boc-pushpa/boc.csv


In [70]:
d1= pd.read_csv('/kaggle/input/boc-pushpa/boc.csv')
df=pd.DataFrame(d1)
df.rename(columns={'Amount (in cr.)':"Amount"},inplace=True )
df.Day[0]=1
print(df)

    Day  Amount
0     1   12.68
1     7   26.89
2    14   47.09
3    21   72.49
4    28   84.75
5    35   91.81
6    42   97.98
7    49  102.39
8    56  105.64
9    63  107.03
10   70  108.03
11   77  108.23
12   84  108.26


/tmp/ipykernel_30/1692612845.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Day[0]=1
/tmp/ipykernel_30/1692612845.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

In [71]:
'''LAGRANGE'S FORMULA FOR INTERPOLATING VALUES IN BETWEEN'''
def lag(x):
    est=0
    l=len(df.Day)
    for i in range(l):
        f=1
        for j in range(l):
            if i!=j:
                f*=(x-df.Day[j])/(df.Day[i]-df.Day[j])
        est+=f*df.Amount[i]
    return est
x=int(input('Enter day for which you want to predict amount: '))
print('Amount collected on',x,'th day is: ', lag(x))

Enter day for which you want to predict amount:  24


Amount collected on 24 th day is:  79.07837467757584


In [72]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
boi=pd.DataFrame(pd.read_csv('/kaggle/input/boxoffice24/boc24.csv'))
X=boi[['firstday','budget','ratings']]
y=boi.collection
v=boi.Verdict

In [73]:
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)
print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 86


In [74]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
l=[15,16,17,18,19,20,21]
s=pd.Series()
for max_leaf_nodes in l:
    my=get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    s[my]=max_leaf_nodes
best_tree_size =s[s.index.min()]
rf_model_on_full_data = RandomForestRegressor(max_leaf_nodes=best_tree_size,random_state=1)
rf_model_on_full_data.fit(X,y)

RandomForestRegressor(max_leaf_nodes=17, random_state=1)

In [75]:
p=pd.DataFrame({'firstday':[274.6],'budget':[400],'ratings':[6.5]})
print(p)
test_preds = rf_model_on_full_data.predict(p)
print(test_preds)

   firstday  budget  ratings
0     274.6     400      6.5
[872.688]
